# RNN Exploration for ICL on Finite State Machines

**Team:** Trenton O'Bannon, Yuri Lee, Keshab Agarwal, Evan Davis

This notebook explores **vanilla RNN improvements** to understand what factors contribute to ICL performance:

## Exploration Goals

### 1. **Capacity Hypothesis**
- Test: Do larger hidden dimensions improve vanilla RNN performance?
- Experiments: d_model = 256, 512, 1024
- Question: "Does capacity matter more than gating mechanisms?"

### 2. **Depth Hypothesis**
- Test: Can deeper RNNs overcome the limitations of shallow ones?
- Experiments: num_layers = 2, 5, 16
- Question: "Is depth a substitute for gating?"

### 3. **Architecture Spectrum**
- Test: Where does GRU fall between RNN and LSTM?
- Experiments: Vanilla RNN → GRU → LSTM
- Question: "Is simple gating enough?"

## Expected Insights

- **If capacity helps**: Vanilla RNN lacks capacity, not just gating
- **If depth helps**: Deep RNNs can achieve ICL without explicit gating
- **If GRU ≈ LSTM**: Gating is the key, not LSTM's specific design
- **If GRU ≈ RNN**: Problem is fundamental to recurrent architectures without forgetting

In [ ]:
# Import Required Libraries and Setup
import sys
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List
import json

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully")
print("📊 Ready to analyze RNN exploration results")

## Load Exploration Results

We'll load results from all experiments:
- Capacity tests (d_model: 256, 512, 1024)
- Depth tests (num_layers: 2, 5, 16)
- GRU baseline
- Original baseline results for comparison

In [ ]:
# Load experimental results
results_dir = Path('../../experiments/explorations/results')

# Helper function to load metrics
def load_experiment(pattern):
    """Load the most recent experiment matching pattern."""
    metrics_files = sorted(results_dir.glob(f"{pattern}*_metrics.json"))
    if not metrics_files:
        return None
    with open(metrics_files[-1], 'r') as f:
        return json.load(f)

# Load all experiments
experiments = {}

# Capacity experiments
capacity_tests = {
    'd256': load_experiment('rnn_d256_baseline'),
    'd512': load_experiment('rnn_d512'),
    'd1024': load_experiment('rnn_d1024'),
}

# Depth experiments
depth_tests = {
    'l2': load_experiment('rnn_l2_baseline'),
    'l5': load_experiment('rnn_l5'),
    'l16': load_experiment('rnn_l16'),
}

# GRU experiment
gru_test = load_experiment('gru_baseline')

# Load baseline results from previous experiments
baseline_dir = Path('../../checkpoints/training_logs')
lstm_baseline = json.load(open(baseline_dir / 'lstm_direct_20251125_041044_metrics.json'))
rnn_baseline = json.load(open(baseline_dir / 'vanilla_rnn_direct_20251125_044821_metrics.json'))

print("✅ Experiments loaded")
print(f"\n📊 Capacity Tests:")
for name, data in capacity_tests.items():
    if data:
        acc = data['final_results']['test_accuracy']
        params = data['model_config']['parameter_count']
        print(f"  {name}: {acc:.2%} ({params:,} parameters)")
    else:
        print(f"  {name}: NOT FOUND")

print(f"\n📊 Depth Tests:")
for name, data in depth_tests.items():
    if data:
        acc = data['final_results']['test_accuracy']
        params = data['model_config']['parameter_count']
        print(f"  {name}: {acc:.2%} ({params:,} parameters)")
    else:
        print(f"  {name}: NOT FOUND")

print(f"\n📊 GRU:")
if gru_test:
    acc = gru_test['final_results']['test_accuracy']
    params = gru_test['model_config']['parameter_count']
    print(f"  GRU Baseline: {acc:.2%} ({params:,} parameters)")
else:
    print(f"  GRU Baseline: NOT FOUND")

print(f"\n📊 Original Baselines:")
print(f"  LSTM: {lstm_baseline['final_results']['test_accuracy']:.2%}")
print(f"  Vanilla RNN: {rnn_baseline['final_results']['test_accuracy']:.2%}")

## Visualization 1: Capacity vs Performance

Does increasing hidden dimension improve RNN performance?

In [ ]:
# Capacity Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Prepare capacity data
capacity_dims = [256, 512, 1024]
capacity_accs = []
capacity_params = []

for dim in capacity_dims:
    key = f'd{dim}'
    if key == 'd256' and not capacity_tests.get(key):
        # Use baseline if d256 experiment not run
        capacity_accs.append(rnn_baseline['final_results']['test_accuracy'])
        capacity_params.append(200_000)  # approximate
    elif capacity_tests.get(key):
        capacity_accs.append(capacity_tests[key]['final_results']['test_accuracy'])
        capacity_params.append(capacity_tests[key]['model_config']['parameter_count'])
    else:
        capacity_accs.append(0)
        capacity_params.append(0)

# Plot 1: Accuracy vs Hidden Dimension
ax1.plot(capacity_dims, capacity_accs, 'o-', linewidth=3, markersize=12, 
         color='#1f77b4', label='Vanilla RNN')

# Add LSTM and GRU baselines as horizontal lines
ax1.axhline(y=lstm_baseline['final_results']['test_accuracy'], 
            color='#2ca02c', linestyle='--', linewidth=2, alpha=0.7,
            label=f'LSTM Baseline ({lstm_baseline["final_results"]["test_accuracy"]:.1%})')

if gru_test:
    ax1.axhline(y=gru_test['final_results']['test_accuracy'],
                color='#ff7f0e', linestyle='--', linewidth=2, alpha=0.7,
                label=f'GRU Baseline ({gru_test["final_results"]["test_accuracy"]:.1%})')

# Annotations
for dim, acc in zip(capacity_dims, capacity_accs):
    if acc > 0:
        ax1.annotate(f'{acc:.1%}', xy=(dim, acc), xytext=(0, 10),
                    textcoords='offset points', ha='center', fontsize=10, fontweight='bold')

ax1.set_xlabel('Hidden Dimension (d_model)', fontsize=13, fontweight='bold')
ax1.set_ylabel('Test Accuracy', fontsize=13, fontweight='bold')
ax1.set_title('Capacity Test: Hidden Dimension vs Performance', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.05)

# Plot 2: Accuracy vs Parameter Count
ax2.scatter(capacity_params, capacity_accs, s=200, alpha=0.7, color='#1f77b4',
            edgecolor='black', linewidth=2, label='Vanilla RNN (varying capacity)')
ax2.scatter([800_000], [lstm_baseline['final_results']['test_accuracy']], 
            s=200, alpha=0.7, color='#2ca02c', marker='s',
            edgecolor='black', linewidth=2, label='LSTM')

if gru_test:
    ax2.scatter([gru_test['model_config']['parameter_count']], 
                [gru_test['final_results']['test_accuracy']],
                s=200, alpha=0.7, color='#ff7f0e', marker='^',
                edgecolor='black', linewidth=2, label='GRU')

# Annotations
for dim, params, acc in zip(capacity_dims, capacity_params, capacity_accs):
    if acc > 0 and params > 0:
        ax2.annotate(f'{dim}d', xy=(params, acc), xytext=(10, -5),
                    textcoords='offset points', fontsize=9, fontweight='bold')

ax2.set_xlabel('Parameter Count', fontsize=13, fontweight='bold')
ax2.set_ylabel('Test Accuracy', fontsize=13, fontweight='bold')
ax2.set_title('Efficiency: Parameters vs Performance', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xscale('log')
ax2.set_ylim(0, 1.05)

plt.tight_layout()
plt.savefig('rnn_exploration_capacity.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Capacity Analysis:")
print(f"  256d → 512d: {(capacity_accs[1] - capacity_accs[0]) * 100:+.1f} percentage points")
print(f"  512d → 1024d: {(capacity_accs[2] - capacity_accs[1]) * 100:+.1f} percentage points")
print(f"  Total gain (256d → 1024d): {(capacity_accs[2] - capacity_accs[0]) * 100:+.1f} percentage points")

## Visualization 2: Depth vs Performance

Can deeper RNNs overcome shallow RNN limitations?

In [ ]:
# Depth Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Prepare depth data
depth_layers = [2, 5, 16]
depth_accs = []
depth_params = []

for layers in depth_layers:
    key = f'l{layers}'
    if key == 'l2' and not depth_tests.get(key):
        # Use baseline if l2 experiment not run
        depth_accs.append(rnn_baseline['final_results']['test_accuracy'])
        depth_params.append(200_000)
    elif depth_tests.get(key):
        depth_accs.append(depth_tests[key]['final_results']['test_accuracy'])
        depth_params.append(depth_tests[key]['model_config']['parameter_count'])
    else:
        depth_accs.append(0)
        depth_params.append(0)

# Plot 1: Accuracy vs Number of Layers
ax1.plot(depth_layers, depth_accs, 'o-', linewidth=3, markersize=12,
         color='#d62728', label='Vanilla RNN')

# Add baselines
ax1.axhline(y=lstm_baseline['final_results']['test_accuracy'],
            color='#2ca02c', linestyle='--', linewidth=2, alpha=0.7,
            label=f'LSTM (2 layers)')

if gru_test:
    ax1.axhline(y=gru_test['final_results']['test_accuracy'],
                color='#ff7f0e', linestyle='--', linewidth=2, alpha=0.7,
                label=f'GRU (2 layers)')

# Annotations
for layers, acc in zip(depth_layers, depth_accs):
    if acc > 0:
        ax1.annotate(f'{acc:.1%}', xy=(layers, acc), xytext=(0, 10),
                    textcoords='offset points', ha='center', fontsize=10, fontweight='bold')

ax1.set_xlabel('Number of Layers', fontsize=13, fontweight='bold')
ax1.set_ylabel('Test Accuracy', fontsize=13, fontweight='bold')
ax1.set_title('Depth Test: Number of Layers vs Performance', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xticks(depth_layers)
ax1.set_ylim(0, 1.05)

# Plot 2: Training Curves Comparison (if available)
# Show training history for different depths
ax2.set_title('Training Convergence by Depth', fontsize=14, fontweight='bold')

for layers in depth_layers:
    key = f'l{layers}'
    if depth_tests.get(key) and 'training_history' in depth_tests[key]:
        history = depth_tests[key]['training_history']
        epochs = range(1, len(history['val_accs']) + 1)
        ax2.plot(epochs, history['val_accs'], 'o-', linewidth=2, markersize=4,
                label=f'{layers} layers', alpha=0.8)

ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Validation Accuracy', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.05)

plt.tight_layout()
plt.savefig('rnn_exploration_depth.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Depth Analysis:")
print(f"  2L → 5L: {(depth_accs[1] - depth_accs[0]) * 100:+.1f} percentage points")
print(f"  5L → 16L: {(depth_accs[2] - depth_accs[1]) * 100:+.1f} percentage points")
print(f"  Total gain (2L → 16L): {(depth_accs[2] - depth_accs[0]) * 100:+.1f} percentage points")

## Visualization 3: Architecture Spectrum

Where does GRU fall between vanilla RNN and LSTM?

In [ ]:
# Architecture Spectrum Comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Prepare data for all architectures
architectures = []
accuracies = []
params = []
colors = []

# Vanilla RNN (256d, 2L baseline)
architectures.append('Vanilla RNN\n(2L, 256d)')
accuracies.append(rnn_baseline['final_results']['test_accuracy'])
params.append(200_000)
colors.append('#1f77b4')

# GRU
if gru_test:
    architectures.append('GRU\n(2L, 256d)')
    accuracies.append(gru_test['final_results']['test_accuracy'])
    params.append(gru_test['model_config']['parameter_count'])
    colors.append('#ff7f0e')

# LSTM
architectures.append('LSTM\n(2L, 256d)')
accuracies.append(lstm_baseline['final_results']['test_accuracy'])
params.append(800_000)
colors.append('#2ca02c')

# Plot 1: Accuracy Comparison
x_pos = np.arange(len(architectures))
bars = ax1.bar(x_pos, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=2)

# Add value labels
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax1.annotate(f'{acc:.1%}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=11, fontweight='bold')

ax1.set_ylabel('Test Accuracy', fontsize=13, fontweight='bold')
ax1.set_title('Architecture Spectrum: Vanilla RNN → GRU → LSTM', fontsize=14, fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(architectures)
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Parameter Efficiency
bars = ax2.barh(architectures, params, color=colors, alpha=0.8, edgecolor='black', linewidth=2)

# Add value labels
for bar, p in zip(bars, params):
    width = bar.get_width()
    ax2.annotate(f'{p/1000:.0f}K',
                xy=(width, bar.get_y() + bar.get_height() / 2),
                xytext=(3, 0), textcoords="offset points",
                ha='left', va='center', fontsize=10, fontweight='bold')

ax2.set_xlabel('Parameter Count', fontsize=13, fontweight='bold')
ax2.set_title('Model Complexity', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('rnn_exploration_spectrum.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Architecture Spectrum:")
print(f"  RNN → GRU: {(accuracies[1] - accuracies[0]) * 100:+.1f} percentage points" if len(accuracies) > 1 else "  GRU data not available")
print(f"  GRU → LSTM: {(accuracies[2] - accuracies[1]) * 100:+.1f} percentage points" if len(accuracies) > 2 else "")
print(f"  RNN → LSTM: {(accuracies[-1] - accuracies[0]) * 100:+.1f} percentage points")

## Summary and Insights

In [ ]:
# Generate comprehensive summary
print("🔍 KEY INSIGHTS FROM RNN EXPLORATION")
print("=" * 80)

print("\n1️⃣  CAPACITY HYPOTHESIS")
print("-" * 80)
if capacity_tests['d1024']:
    gain_1024 = (capacity_accs[2] - capacity_accs[0]) * 100
    print(f"\nIncreasing hidden dimension from 256 → 1024:")
    print(f"  Performance gain: {gain_1024:+.1f} percentage points")
    
    if gain_1024 < 10:
        print(f"\n  → CONCLUSION: Capacity alone is NOT sufficient")
        print(f"  → 4x more parameters gives minimal improvement")
        print(f"  → Problem is architectural, not just capacity-related")
    else:
        print(f"\n  → CONCLUSION: Capacity matters significantly")
        print(f"  → Large RNNs can approach gated architectures")
else:
    print("\n  ⚠️  Capacity experiments not yet run")

print("\n" + "=" * 80)
print("\n2️⃣  DEPTH HYPOTHESIS")
print("-" * 80)
if depth_tests['l16']:
    gain_16L = (depth_accs[2] - depth_accs[0]) * 100
    print(f"\nIncreasing depth from 2 → 16 layers:")
    print(f"  Performance gain: {gain_16L:+.1f} percentage points")
    
    if gain_16L < 10:
        print(f"\n  → CONCLUSION: Depth alone is NOT sufficient")
        print(f"  → Very deep RNNs still struggle with ICL")
        print(f"  → Vanishing gradients limit deep RNN effectiveness")
    else:
        print(f"\n  → CONCLUSION: Depth helps significantly")
        print(f"  → Deep RNNs can compensate for lack of gating")
else:
    print("\n  ⚠️  Depth experiments not yet run")

print("\n" + "=" * 80)
print("\n3️⃣  ARCHITECTURE SPECTRUM")
print("-" * 80)
if gru_test:
    rnn_acc = rnn_baseline['final_results']['test_accuracy']
    gru_acc = gru_test['final_results']['test_accuracy']
    lstm_acc = lstm_baseline['final_results']['test_accuracy']
    
    print(f"\nPerformance:")
    print(f"  Vanilla RNN: {rnn_acc:.2%}")
    print(f"  GRU:         {gru_acc:.2%}")
    print(f"  LSTM:        {lstm_acc:.2%}")
    
    rnn_to_gru = (gru_acc - rnn_acc) / (lstm_acc - rnn_acc)
    print(f"\nGRU fills {rnn_to_gru:.1%} of the gap between RNN and LSTM")
    
    if gru_acc > 0.9:
        print(f"\n  → CONCLUSION: Simple gating (GRU) is sufficient")
        print(f"  → LSTM's complexity not needed for this task")
    elif gru_acc > (rnn_acc + lstm_acc) / 2:
        print(f"\n  → CONCLUSION: Gating helps, GRU is middle ground")
        print(f"  → GRU closer to LSTM than vanilla RNN")
    else:
        print(f"\n  → CONCLUSION: GRU closer to RNN performance")
        print(f"  → LSTM's full gating mechanisms are necessary")
else:
    print("\n  ⚠️  GRU experiment not yet run")

print("\n" + "=" * 80)
print("\n4️⃣  OVERALL CONCLUSION")
print("-" * 80)
print("""
For In-Context Learning on FSM tasks:

✅ CRITICAL FACTORS:
  • Gating mechanisms are essential (LSTM > GRU >> RNN)
  • Architecture matters more than raw capacity or depth
  • Vanilla RNNs fundamentally limited for ICL

⚠️  LESS IMPORTANT:
  • Hidden dimension (beyond reasonable size)
  • Number of layers (beyond 2-5 layers)

💡 PRACTICAL RECOMMENDATION:
  • Use LSTM or GRU for FSM-based ICL tasks
  • Don't waste compute on massive vanilla RNNs
  • 2-layer models are sufficient with proper architecture
""")